## Deep Fake
- 學習 Deep fake 原理
- 學習如何使用 keras 實做 Deep fake, 試著想出反制方法。
- 想一個 Deep fake 應用場景
- 產品化

### Deepfake 介紹
- 介紹:
    - 利用深度學習將人臉交換，通常用於video。
    - 


### *資料處理
- 將影片轉化為圖片檔案

### 使用 autoencoder 建立模型


In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf

In [2]:
# datasets creation and splitting

def create_dataset(path):
    images = []
    for diraname, _, filnames in os.walk(path):
        for filename in filenames:
            image = cv2.imread(os.path.join(dirname, filename))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = image.astype('float32')
            image /= 255.0
            images.append(image)
    images = np.array(images)
    return images

In [3]:
# faces_1 = create_dataset('/kaggle/input/presidentsfacesdataset/trump/')
# faces_2 = create_dataset('/kaggle/input/presidentsfacesdataset/biden/')

In [4]:
# X_train_a, X_test_a, y_train_a, y_test_a = train_test_split(faces_1, faces_1, test_size=0.20, random_state=0)
# X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(faces_2, faces_2, test_size=0.15, random_state=0)

In [22]:
# autuencoders 建立

def get_encoder():
    input_img = layers.Input(shape=(120, 120, 3))
    x = layers.Conv2D(256,kernel_size=5, strides=2, padding='same',activation='relu')(input_img)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(512,kernel_size=5, strides=2, padding='same',activation='relu')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(1024,kernel_size=5, strides=2, padding='same',activation='relu')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Flatten()(x)
    x = layers.Dense(3*3*1024)(x)
    encoded = layers.Reshape((3,3,1024))(x)
    encoder = keras.Model(input_img, encoded,name="encoder")    
    return encoder

In [25]:
encoder = get_encoder()
encoder.summary()

# del encoder   

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 120, 120, 3)]     0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 60, 60, 256)       19456     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 30, 30, 256)       0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 15, 15, 512)       3277312   
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 8, 8, 512)         0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 4, 4, 1024)        13108224  
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 2, 2, 1024)        0   

In [26]:
# decoder

def get_decoder():
    decoder_input= layers.Input(shape=((3,3,1024)))
    x = layers.Conv2D(1024,kernel_size=5, strides=2, padding='same',activation='relu')(decoder_input)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(512,kernel_size=5, strides=2, padding='same',activation='relu')(x)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(256,kernel_size=5, strides=2, padding='same',activation='relu')(x)
    x = layers.Flatten()(x)
    x = layers.Dense(np.prod((120, 120, 3)))(x)
    decoded = layers.Reshape((120, 120, 3))(x)
    decoder = keras.Model(decoder_input, decoded,name="decoder")
    return decoder

decoder = get_decoder()
decoder.summary()

# del decoder

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 3, 3, 1024)]      0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 2, 2, 1024)        26215424  
_________________________________________________________________
up_sampling2d_11 (UpSampling (None, 4, 4, 1024)        0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 2, 2, 512)         13107712  
_________________________________________________________________
up_sampling2d_12 (UpSampling (None, 4, 4, 512)         0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 2, 2, 256)         3277056   
_________________________________________________________________
flatten_11 (Flatten)         (None, 1024)              0   

In [27]:
# Autoencoder
# 得到一個基本單位的autoencoder, 接下來只要創建2個, 且訓練好即可做swap!

def get_autoencoder():
    auto_input = layers.Input(shape=(120, 120, 3))
    encoded = encoder(auto_input)
    decoded = decoder(encoded)
    
    autoencoder = keras.Model(auto_input, decoded, name='autoencoder')
    autoencoder.compile(optimizer=keras.optimizers.Adam(lr=5e-5, beta_1=0.5, beta_2=0.999), loss='mse')
    autoencoder.summary()
    return autoencoder

autoencoder = get_autoencoder()

Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 120, 120, 3)]     0         
_________________________________________________________________
encoder (Functional)         (None, 3, 3, 1024)        54162944  
_________________________________________________________________
decoder (Functional)         (None, 120, 120, 3)       86880192  
Total params: 141,043,136
Trainable params: 141,043,136
Non-trainable params: 0
_________________________________________________________________


C:\Users\aband\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [47]:
# swapping encoder and decoders in Keras


# # TO LOAD ONLY THE ENCODER A
# encoder_a = keras.Model(autoencoder_a.layers[1].input, autoencoder_a.layers[1].output)
# # TO LOAD ONLY THE DECODER A
# decoder_a = keras.Model(autoencoder_a.layers[2].input, autoencoder_a.layers[2].output)
# # TO LOAD ONLY THE ENCODER B
# encoder_b = keras.Model(autoencoder_b.layers[1].input, autoencoder_b.layers[1].output)
# # TO LOAD ONLY THE DECODER B
# decoder_b = keras.Model(autoencoder_b.layers[2].input, autoencoder_b.layers[2].output)
 
# # TO TRANSFORM SRC IMAGES
# input_test = encoder_a.predict(np.array([X_test_a[30]]))
# output_test = decoder_b.predict(input_test)
 
# # TO TRANSFORM DST IMAGES
# input_test = encoder_b.predict(np.array([X_test_b[30]]))
# output_test = decoder_a.predict(input_test)